In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirpath, _, filenames in os.walk('/kaggle/input'):
#dirnames is not needed, so any insignificant temp var can be represented as _
    for filename in filenames:
        print(os.path.join(dirpath, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
#help(os.walk)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing, linear_model, feature_extraction

In [3]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [4]:
df_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,7613.0,5441.934848,3137.11609,1.0,2734.0,5408.0,8146.0,10873.0
target,7613.0,0.429660,0.49506,0.0,0.0,0.0,1.0,1.0


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
df_train.iloc[:3]['text'].values

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"],
      dtype=object)

In [7]:
df_train[df_train['target']==1]['text'].iloc[:2]
df_train[df_train['location'].notnull()]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


In [8]:
print(' '.join(df_train['text'][0].split(' ')[:3]))

Our Deeds are


In [9]:
df_train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [10]:
# Although Tfidfvectorizer is much simpler, I will use Countvectorizer and TfidfTransformer in lieu 
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

counter = CountVectorizer()
tfidfer = TfidfTransformer()

#Just so that the counter instance 'learns' the vocab inside df_train
X_train_counts = counter.fit_transform(df_train['text'])
X_test = counter.transform(df_test['text'])
tfidfer.fit(X_train_counts)
X_test = tfidfer.transform(X_test)

In [11]:
# #Ensure that the train and test vectors have the same set of tokens.
# #Token is a smallest lexical unit - article, word etc. 
# X_train = vectorizer.fit_transform(df_train['text'])
# #To avoid data leakage, we only transform the test_df['text']
# X_test = vectorizer.transform(df_test['text'])

In [12]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [13]:
X_train = df_train['text']
y_train = df_train['target']
pipe_mnnb = Pipeline([('counter', CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MultinomialNB())])
pipe_mnnb = pipe_mnnb.fit(X_train, y_train)

In [14]:
score = model_selection.cross_val_score(pipe_mnnb, X_train, y_train, scoring='f1', cv=5)
print(score.mean())

0.6406922913879345


In [15]:
df_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [16]:
pipe_svc = Pipeline([('counter', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', SVC())])
pipe_svc = pipe_svc.fit(X_train,y_train)
score_svc = model_selection.cross_val_score(pipe_svc, X_train, y_train,scoring='f1',cv=5)
print(score_svc)

[0.62969925 0.60759494 0.65463495 0.61694291 0.75176194]


In [17]:
#Run once, save the best_params.
param_svc = {
    'counter__ngram_range':[(1,1),(1,2)],
    'tfidf__use_idf':[True, False],
    'clf__C':[0.0001,0.001,0.01,0.1,1],
    'clf__kernel':['linear','poly','rbf'],
    'clf__degree':[1,2,3]
}
pipe_svc = Pipeline([('counter', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', SVC())])
pipe_svc.fit(X_train,y_train)
searcher_svc = GridSearchCV(pipe_svc, param_svc, scoring='f1', n_jobs=-1, refit=False, cv=10)
#searcher_svc.fit(X_train, y_train)
#print(searcher_svc.best_params_)
#score_svc = model_selection.cross_val_score(searcher, X_train, y_train, scoring='f1', cv=10)
#print(score_svc)

#Best params
#{'clf__C': 1, 'clf__degree': 1, 'clf__kernel': 'linear', 'counter__ngram_range': (1, 2), 'tfidf__use_idf': True}


In [18]:
best_param_svc={'clf__C': 1, 'clf__degree': 1, 'clf__kernel': 'linear', 'counter__ngram_range': (1, 2), 'tfidf__use_idf': True}

In [19]:
#Use idf is obviously True by default, but just to be explicit
pipe_svc = Pipeline([('counter', CountVectorizer(ngram_range=(1,2))),('tfidf', TfidfTransformer(use_idf=True)), ('clf', SVC(C=1, degree=1, kernel='linear'))])
pipe_svc = pipe_svc.fit(X_train,y_train)
score_svc = model_selection.cross_val_score(pipe_svc, X_train, y_train,scoring='f1',cv=5)
print(score_svc)

[0.66103128 0.60993976 0.67913239 0.65714286 0.74766355]


In [20]:
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [21]:
submission['target'] = pipe_svc.predict(df_test['text'])

In [22]:
# submission = submission.reset_index(drop=True)

In [23]:
submission.to_csv('submission.csv', index=False)

In [24]:
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
